# ___**决策树**___
### 基本原理
> - 从前面的kNN算法的基本原理和实现来看，kNN算法的最大/最显著的缺点是无法给出数据的内在含义，而这也是决策树的主要优势，其数据形式非常易于理解。
> - 决策树的一个重要任务是为了理解数据中所蕴含的知识信息，因此决策树可以使用不熟悉的数据集合，并从中提取出一系列规则，而这些机器根据数据集创建规则的过程，就是机器学习的过程
> - 决策树也是一个典型的监督学习算法
> - 简要来说，构建决策树的过程就是将现有数据不断划分，直到每个划分出的数据子集中的数据归于的类型相同。
### 数据划分、信息增益和熵
> - 划分数据集的大原则是: 将无序的数据变得更加有序
> - 在划分数据增益前后信息发生的变化称为信息增益，以某个特征值划分数据后获得信息增益最大时，该特征值就是最好的选择
> - 集合信息的度量方式称为香农熵或者熵(采用)，熵定义为信息的期望值
- 计算给定数据集的熵

In [ ]:
from math import log
def clacShannonEnt(dataSet):
    """
    计算给定数据集的熵(香农熵)
    :param dataSet: 
    :return: 香农熵
    """
    # 获取数据集数据的数量
    numEntries = len(dataSet)
    # 创建空的标签字典，用于存储标签及该标签的出现次数
    labelCounts = {}
    # 遍历数据集中的每一个数据样本
    for featVec in dataSet:
        # 获得当前数据样本的标签
        currentLabel = featVec[-1]
        # 如果这个标签在标签字典中不存在
        if currentLabel not in labelCounts.keys():
            # 则初始化这个标签在字典中的值
            labelCounts[currentLabel] = 0
        # 更新标签出现的次数
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        # 计算在数据集中每个标签出现的概率，就是该标签数量除以样本总数量
        prob = float(labelCounts[key]) / numEntries
        # 香农熵的累加，原始公式为-Σ(总样本数) 该标签出现的概率*该概率的底数为2的对数
        shannonEnt -= prob * log(prob, 2)
    # 熵越高，代表着混合的数据越多，数据越难以区分，可以通过增加额外的标签来进一步区分数据以降低熵
    return shannonEnt

### ___**此外，另一个度量信息无序程度的方法是基尼不纯度(Gini impurity)，简单来说就是从一个数据集中随机选取子项，度量其被错误分类到其他分组里的概率**___

### 划分数据集

In [2]:
def splitDataSet(dataSet, axis, value):
    """
    划分数据集
    :param dataSet: 待划分的数据集
    :param axis:    划分数据集的特征
    :param value:   需要返回的特征的值
    :return:        划分后的数据集
    """
    # 创建空列表用于存储划分后的数据集
    retDataSet = []
    for featVec in dataSet:
        # 如果该样本的特征值与需要返回的特征值相同，则将该样本抽取出来划分到新的数据集中
        if featVec[axis] == value:
            # 切片获取从列表开头到axis-1的元素
            reducedFeatVec = featVec[:axis]
            # 切片追加从axis+1到列表末尾的元素，现在列表reducedFeatVec包含了除axis索引以外所有的元素
            reducedFeatVec.extend(featVec[axis+1:])
            # 追加到划分后的数据集中
            retDataSet.append(reducedFeatVec)
    return retDataSet

### 选择最好的数据集划分方式

In [ ]:
from math import log
from 决策树.Decision_Tree import calcShannonEnt, majorityCnt

def chooseBestFeatureToSplit(dataSet):
    """
    选择最好的数据集划分方式
    :param dataSet: 原始数据集
    :return:        最佳划分特征
    """
    # 获取数据集的特征数量
    numFeatures = len(dataSet[0]) - 1
    # 获取原始数据集的熵
    baseEntropy = calcShannonEnt(dataSet)
    # 声明最佳信息增益和最佳特征索引
    bestInfoGain = 0.0
    bestFeature = -1
    # 遍历所有特征
    for i in range(numFeatures):
        # 使用条件for获取一个样本的所有特征值列表
        featList = [example[i] for example in dataSet]
        # 将特征值列表转换为集合，用于去除重复特征值
        uniqueVals = set(featList)
        # 声明划分数据后的熵
        newEntropy = 0.0
        # 遍历特征集合中的每个特征值
        for value in uniqueVals:
            # 划分数据集
            subDataSet = splitDataSet(dataSet, i, value)
            # 计算划分后的数据集的熵
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        # 计算信息增益
        infoGain = baseEntropy - newEntropy
        # 如果信息增益大于最佳信息增益，则更新最佳信息增益和最佳特征
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

# 递归构建决策树
- #### 递归的终止条件: 程序遍历完所有划分数据集的属性，或者每个分支下的所有实例都具有相同的分类。

In [ ]:
def createTree(dataSet, labels):
    """
    递归构建决策树
    :param dataSet: 数据集
    :param labels:  标签集，算法本身不需要这个变量，但是为了给出数据明确的含义，需要给出标签机
    :return: 
    """
    # 获得数据集的所有特征标签
    classList = [example[-1] for example in dataSet]
    # 停止条件1: 如果数据集中所有实例属于同一类别，直接返回该类别
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 停止条件2: 如果使用完了所有特征仍不能将数据集划分成仅包含唯一类别的分组，则返回出现次数最多的类别
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 获得当前数据集的最佳划分特征
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    # 创建空嵌套字典用于存储子树
    myTree = {bestFeatLabel: {}}
    # 去除标签列表中的最佳特征
    del labels[bestFeat]
    # 获得数据集的所有特征标签
    featValues = [example[bestFeat] for example in dataSet]
    # 去除重复的特征值
    uniqueVals = set(featValues)
    # 遍历特征值集合中的每个特征
    for value in uniqueVals:
        # 获得标签集的所有标签
        subLabels = labels[:]
        # 递归构建子树: 最优划分特征下的每个值都对应一个子树
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    # 返回子树
    return myTree